In [20]:
import astropy.units as u
from astropy.coordinates.sky_coordinate import SkyCoord
from astropy.units import Quantity
from astroquery.gaia import Gaia
from astroquery.simbad import Simbad
from astropy.time import Time
from astropy.io import ascii

In [2]:
import matplotlib.pyplot as plt
import numpy as np

# Suppress warnings. Comment this out if you wish to see the warning messages
#import warnings
#warnings.filterwarnings('ignore')

In [3]:
sample_table = ascii.read('sample_final.csv')
sample_table

hostname,pl_name,sy_dist,rastr,decstr,sy_kmag,st_mass,st_rad,st_raderr1,st_raderr2,st_teff,pl_tranmid,pl_tranmiderr1,pl_tranmiderr2,pl_rade,pl_radeerr1,pl_radeerr2,pl_bmasse,pl_bmasseerr1,pl_bmasseerr2,pl_orbper,pl_orbpererr1,pl_orbpererr2,pl_orbsmax,pl_orbincl,pl_orbeccen,pl_orbeccenerr1,pl_orbeccenerr2,tran_flag,rv_flag,pl_rp_rs,pl_g,pl_teq_a0,pl_s,pl_rho,filter,subarray,nobs,ngroup,frac_fullwell
str9,str11,float64,str12,str13,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,float64,float64,float64,float64,float64,str6,str6,int64,int64,float64
GJ 3473,GJ 3473 b,27.3644,08h02m22.47s,+03d20m13.79s,8.829001,0.36,0.36,0.01,-0.01,3347.0,2458492.20412163,0.00037015,-0.00037015,1.264,0.05,-0.049,1.86,0.3,-0.3,1.198003975976397,5.76582216e-07,-5.76582216e-07,0.01589,87.95,0.0,--,--,1,1,0.0321894750291473,11.407064002554433,768.2070915703122,57.83231621613784,5.06105605260494,f1500w,sub256,4,39,0.65
GJ 357,GJ 357 b,9.44181,09h36m01.79s,-21d39m54.23s,6.475,0.34,0.34,0.02,-0.02,3505.0,2458517.998679131,0.0003765,-0.0003765,1.217,0.084,-0.083,1.84,0.31,-0.31,3.9306063296116895,1.16151077e-06,-1.16151077e-06,0.035,89.12,--,--,--,1,1,0.03281564780289002,12.17283537042799,526.7769415618036,12.786873373040743,5.609388414817885,f1500w,sub64,1,21,0.8
HD 260655,HD 260655 b,10.0055,06h37m09.97s,+17d33m58.57s,5.862,0.44,0.439,0.003,-0.003,3803.0,2459497.9102,0.0003,-0.0003,1.24,0.023,-0.023,2.14,0.34,-0.34,2.76953,3e-05,-3e-05,0.02933,87.35,0.039,0.043,-0.023,1,1,0.02589563026767422,13.63720864077306,709.4729664292033,42.072735108287475,6.167627593216154,f1500w,sub64,2,16,0.95
L 98-59,L 98-59 c,10.6194,08h18m07.89s,-68d18m52.08s,7.101,0.27,0.3,0.03,-0.02,3415.0,2458367.27375,0.00013,-0.00022,1.34,0.07,-0.07,2.42,0.35,-0.34,3.6906777,1.6e-06,-2.6e-06,0.0304,88.11,0.103,0.045,-0.058,1,1,0.04094990177758612,13.20568249247054,517.3063880405923,11.891830329088151,5.5267573282965365,f1500w,sub128,1,23,0.7
LHS 1140,LHS 1140 c,14.9861,00h44m59.67s,-15d16m26.79s,8.821,0.19,0.21,0.0035,-0.0035,2988.0,2458389.2942323,0.00062395,-0.00062395,1.169,0.037,-0.038,1.76,0.17,-0.16,3.777939728,5.83523921e-06,-5.83523921e-06,0.02734,89.913,0.274,--,--,1,1,0.05103457908102151,12.61940060872384,399.3229614135901,4.2223523461028085,6.053946228618852,f1500w,sub256,3,36,0.65
LHS 1478,LHS 1478 b,18.2276,02h57m21.33s,+76d33m05.06s,8.767,0.24,0.25,0.01,-0.01,3381.0,2458786.75425,0.00042,-0.00042,1.242,0.051,-0.049,2.33,0.2,-0.2,1.9495378,4e-06,-4.1e-06,0.01848,87.452,--,--,--,1,1,0.045546069857697276,14.800207739484296,599.6495187649439,21.470754020600786,6.682832228969028,f1500w,sub256,2,39,0.65
LTT 3780,LTT 3780 b,21.9814,10h18m34.78s,-11d43m04.08s,8.204001,0.4,0.37,0.01,-0.01,3331.0,2458543.91281,0.00048,-0.00052,1.32,0.06,-0.06,2.47,0.24,-0.24,0.768377,1.4e-06,-1.4e-06,0.01203,85.9,0.064,0.075,-0.046,1,1,0.032707061524655315,13.89006163347313,890.7913247681156,104.55895980434461,5.901257760073781,f1500w,sub256,2,22,0.65
TOI-1468,TOI-1468 b,24.7399,01h06m36.93s,+19d13m29.71s,8.497,0.34,0.34,0.01,-0.01,3496.0,2458765.68079,0.0007,-0.00069,1.28,0.038,-0.039,3.21,0.24,-0.24,1.8805136,2.4e-06,-2.6e-06,0.02102086,88.47,--,--,--,1,1,0.03451440360534036,19.197301030931996,677.9832124158797,35.08593869199558,8.410940260139574,f1500w,sub256,3,31,0.65
TOI-270,TOI-270 b,22.453,04h33m39.86s,-51d57m26.62s,8.251,0.386,0.38,0.008,-0.008,3506.0,2458387.09505,0.00074,-0.00074,1.206,0.039,-0.039,1.58,0.26,-0.26,3.3601538,4.8e-06,-4.8e-06,0.03197,89.39,0.034,0.025,-0.025,1,1,0.029095982841969084,10.64431095709798,582.8619296384281,19.165497515761714,4.949765091265365,f1500w,sub256,4,25,0.65


## Retrieve targets from Gaia and return properties

In [4]:
sample_table = ascii.read('sample_final.csv')

for star in sample_table['hostname']:
    print(star)
    star_names = Simbad.query_objectids(star)
    for name in star_names:
        if 'gaia' in name['ID'].lower() and 'dr3' in name['ID'].lower():
            print(name['ID'])
            query = f"SELECT source_id, ref_epoch, ra, dec, parallax, pmra, pmdec, \
                    distance_gspphot, teff_gspphot, azero_gspphot, phot_g_mean_mag, radial_velocity \
                    FROM gaiadr3.gaia_source \
                    WHERE source_id = {name['ID'].split(' ')[-1]}"

            job     = Gaia.launch_job_async(query)
            results = job.get_results()
            print('    Reference epoch')
            print('   ', results['ref_epoch'].value[0], results['ref_epoch'].unit)
            print('    RA    Dec')
            coordinates = SkyCoord(results['ra'].value[0]*u.deg, results['dec'].value[0]*u.deg, frame='icrs')
            coords_string = coordinates.to_string('hmsdms')
            print(f'   {coords_string}')
            print('    Proper motion RA, Dec', results['pmra'].unit, results['pmdec'].unit)
            print('   ', results['pmra'].value[0]*u.mas/u.yr, results['pmdec'].value[0]*u.mas/u.yr)
            print('    Parallax')
            print('   ', (results['parallax'].value[0]*u.mas).to(u.arcsec))


GJ 3473
Gaia DR3 3094290054327367168
INFO: Query finished. [astroquery.utils.tap.core]
    Reference epoch
    2016.0 yr
    RA    Dec
   08h02m22.45210865s +03d20m13.59632883s
    Proper motion RA, Dec mas / yr mas / yr
    -403.19796827775485 mas / yr -380.9280968279645 mas / yr
    Parallax
    0.03661053015816239 arcsec
GJ 357
Gaia DR3 5664814198431308288
INFO: Query finished. [astroquery.utils.tap.core]
    Reference epoch
    2016.0 yr
    RA    Dec
   09h36m01.7964296s -21d39m54.72307893s
    Proper motion RA, Dec mas / yr mas / yr
    138.7217909625574 mas / yr -990.3417190688432 mas / yr
    Parallax
    0.10597890913558383 arcsec
HD 260655
Gaia DR3 3359074685047632640
INFO: Query finished. [astroquery.utils.tap.core]
    Reference epoch
    2016.0 yr
    RA    Dec
   06h37m09.94355157s +17d33m58.73946387s
    Proper motion RA, Dec mas / yr mas / yr
    -764.4141395476029 mas / yr 337.88323421402026 mas / yr
    Parallax
    0.10002319547433648 arcsec
L 98-59
Gaia DR3 52710552

In [9]:
def get_gaia_dr3_data(star_name):
    print(star_name)
    star_aliases = Simbad.query_objectids(star_name)
    for alias in star_aliases:
        if 'gaia' in alias['ID'].lower() and 'dr3' in alias['ID'].lower():
            print(alias['ID'])
            gaia_source_id = alias['ID'].split(' ')[-1]
            print(gaia_source_id)
            
            query = f"SELECT source_id, ref_epoch, ra, dec, pm, pmra, pmdec, parallax, \
                    distance_gspphot, teff_gspphot, azero_gspphot, phot_g_mean_mag, radial_velocity \
                    FROM gaiadr3.gaia_source \
                    WHERE source_id = {gaia_source_id}"

            job     = Gaia.launch_job_async(query)
            results = job.get_results()
            print('    Reference epoch:')
            print('   ', results['ref_epoch'].value[0], results['ref_epoch'].unit)
            print('    RA    Dec:')
            coordinates = SkyCoord(results['ra'].value[0]*u.deg, results['dec'].value[0]*u.deg, frame='icrs')
            coords_string = coordinates.to_string('hmsdms')
            print(f'   {coords_string}')
            print('    Proper motion RA, Dec:')#, results['pmra'].unit, results['pmdec'].unit)
            print('   ', results['pmra'].value[0]*u.mas/u.yr, results['pmdec'].value[0]*u.mas/u.yr)
            print('    Parallax:')
            print('   ', (results['parallax'].value[0]*u.mas))
            
    return results

In [6]:
get_gaia_dr3_data(sample_table['hostname'][8])

TOI-270
Gaia DR3 4781196115469953024
4781196115469953024
INFO: Query finished. [astroquery.utils.tap.core]
    Reference epoch:
    2016.0 yr
    RA    Dec:
   04h33m39.86381633s -51d57m26.75223444s
    Proper motion RA, Dec:
    83.08204378717042 mas / yr -269.8034870673143 mas / yr
    Parallax:
    44.489913488441964 mas


## Compute proper motions to give target placement in 2024

In [137]:
target = sample_table[3]
gaia_results = get_gaia_dr3_data(target['hostname'])

L 98-59
Gaia DR3 5271055243163629056
5271055243163629056
INFO: Query finished. [astroquery.utils.tap.core]
    Reference epoch:
    2016.0 yr
    RA    Dec:
   08h18m07.89506385s -68d18m52.24672362s
    Proper motion RA, Dec:
    94.79378412168865 mas / yr -340.08431073762046 mas / yr
    Parallax:
    94.26635965165642 mas


In [138]:
gaia_results

source_id,ref_epoch,ra,dec,pm,pmra,pmdec,parallax,distance_gspphot,teff_gspphot,azero_gspphot,phot_g_mean_mag,radial_velocity
,yr,deg,deg,mas / yr,mas / yr,mas / yr,mas,pc,K,mag,mag,km / s
int64,float64,float64,float64,float32,float64,float64,float64,float32,float32,float32,float32,float32
5271055243163629056,2016.0,124.53289609935437,-68.31451297878304,353.04843,94.79378412168865,-340.08431073762046,94.26635965165642,10.6115,3178.8445,0.0031,10.605513,-6.095192


In [139]:
def calculate_proper_motions(gaia_results, future_time='now'):

    ra = float(gaia_results['ra'])       # [deg]
    dec = float(gaia_results['dec'])     # [deg]
    c = SkyCoord(f"{ra} {dec}" , unit='deg', frame='icrs')

    mu_ra = float(gaia_results['pmra']) / 1000   # proper motion ra [as/yr]
    mu_dec = float(gaia_results['pmdec']) / 1000  # proper motion dec [as/yr]
    parallax = float(gaia_results['parallax'])  # mas
    d = 1/(parallax / 1000)       # distance [pc]
    epoch = float(gaia_results['ref_epoch'])         # elapsed time since position was measured [yrs]
    time_now = Time.now()
    if future_time=='now':
        epoch_new = time_now.value.year + time_now.value.month/12  # get year + month of now
    else: epoch_new = future_time
    t = epoch_new - epoch
    v_r = 0          # radial velocity [km/s]  # okay for nearby stuff 

    # convert to transverse (linear) velocities
    v_ra = mu_ra * d * 4.74         #[km/s]
    v_dec = mu_dec * d * 4.74       #[km/s]

    # convert ra and dec into radiansASTRO
    ra = ra*np.pi/180
    dec = dec*np.pi/180

    # convert to cartesian velocities (assuming radial velocity v_r = 0)
    v_x = ((v_r*np.cos(dec)*np.cos(ra)) - (v_ra*np.sin(ra)) - (v_dec*np.sin(dec)*np.cos(ra)))/977780     #[pc/yr]
    v_y = ((v_r*np.cos(dec)*np.sin(ra)) + (v_ra*np.cos(ra)) - (v_dec*np.sin(dec)*np.sin(ra)))/977780     #[pc/yr]
    v_z = ((v_r*np.sin(dec)) + (v_dec*np.cos(dec)))/977780                                               #[pc/yr]

    # use the velocity to transform to current position
    x0 = d * np.cos(dec) * np.cos(ra)
    y0 = d * np.cos(dec) * np.sin(ra)
    z0 = d * np.sin(dec)

    xt = x0 + v_x*t             # [pc]
    yt = y0 + v_y*t             # [pc]
    zt = z0 + v_z*t             # [pc]

    # convert to a new usable ra and dec
    dxy = np.sqrt(xt**2 + yt**2)                  # [pc]
    dec_new = np.arctan(zt/dxy)*180/np.pi         # [deg]
    ra_new = np.arctan(yt/xt)*180/np.pi           # [deg]

    if yt < 0 and xt < 0:
        # ra is in the 3rd quadrant: 180 deg - 270 deg
        ra_new += 180.
    elif yt > 0 and xt < 0:
        # ra is in the 3rd quadrant: 180 deg - 270 deg
        ra_new += 180.


    c_new = SkyCoord(ra_new, dec_new, unit='deg', frame='icrs')

    return epoch_new, c, c_new

In [140]:
def ra_dec_string(ra_dec):
    ra, dec = ra_dec.split(' ')
    ra_new  = ra.replace('h', ':').replace('m', ':').replace('s', '')
    dec_new = dec.replace('d', ':').replace('m', ':').replace('s', '')
    
    return ra_new + ' ' + dec_new

In [143]:
future_time = 2050
epoch_new, c, c_new = calculate_proper_motions(gaia_results, future_time=future_time)

pos_old = ra_dec_string(c.to_string('hmsdms'))
pos_now = ra_dec_string(c_new.to_string('hmsdms'))

starname = target['hostname']
print(f'Positions for {starname}')
print(f'   Position at {float(epoch)}: {pos_old}')
print(f'   Position at {epoch_new:.1f}: {pos_now}')

Positions for L 98-59
   Position at 2016.0: 08:18:07.89506385 -68:18:52.24672362
   Position at 2050.0: 08:18:08.47658193 -68:19:03.80852382


## Compute offsets for targets that need them

In [9]:
sample_table

hostname,pl_name,sy_dist,rastr,decstr,sy_kmag,st_mass,st_rad,st_raderr1,st_raderr2,st_teff,pl_tranmid,pl_tranmiderr1,pl_tranmiderr2,pl_rade,pl_radeerr1,pl_radeerr2,pl_bmasse,pl_bmasseerr1,pl_bmasseerr2,pl_orbper,pl_orbpererr1,pl_orbpererr2,pl_orbsmax,pl_orbincl,pl_orbeccen,pl_orbeccenerr1,pl_orbeccenerr2,tran_flag,rv_flag,pl_rp_rs,pl_g,pl_teq_a0,pl_s,pl_rho,filter,subarray,nobs,ngroup
str9,str11,float64,str12,str13,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,float64,float64,float64,float64,float64,str6,str9,int64,int64
GJ 3473,GJ 3473 b,27.3644,08h02m22.47s,+03d20m13.79s,8.829001,0.36,0.36,0.01,-0.01,3347.0,2458492.20408,0.00043,-0.00042,1.264,0.05,-0.049,1.86,0.3,-0.3,1.1980035,1.8e-06,-1.9e-06,0.01589,87.95,0.0,--,--,1,1,0.0321894750291473,11.407064002554433,768.2070915703122,57.83231621613784,5.06105605260494,f1500w,full,4,7
GJ 357,GJ 357 b,9.44181,09h36m01.79s,-21d39m54.23s,6.475,0.34,0.34,0.02,-0.02,3505.0,2458517.998679131,0.0003765,-0.0003765,1.217,0.084,-0.083,1.84,0.31,-0.31,3.930607947167463,2.51410086e-06,-2.51410086e-06,0.035,89.12,--,--,--,1,1,0.03281564780289002,12.17283537042799,526.7769415618036,12.786873373040743,5.609388414817885,f1500w,sub256,1,8
HD 260655,HD 260655 b,10.0055,06h37m09.97s,+17d33m58.57s,5.862,0.44,0.439,0.003,-0.003,3803.0,2459497.9102,0.0003,-0.0003,1.24,0.023,-0.023,2.14,0.34,-0.34,2.76953,3e-05,-3e-05,0.02933,87.35,0.039,0.043,-0.023,1,1,0.02589563026767422,13.63720864077306,709.4729664292033,42.072735108287475,6.167627593216154,f1500w,sub128,2,12
L 98-59,L 98-59 c,10.6194,08h18m07.89s,-68d18m52.08s,7.101,0.27,0.3,0.03,-0.02,3415.0,2458367.27375,0.00013,-0.00022,1.34,0.07,-0.07,2.42,0.35,-0.34,3.6906777,1.6e-06,-2.6e-06,0.0304,88.11,0.103,0.045,-0.058,1,1,0.04094990177758612,13.20568249247054,517.3063880405923,11.891830329088151,5.5267573282965365,f1500w,sub256,1,13
LHS 1140,LHS 1140 c,14.9861,00h44m59.67s,-15d16m26.79s,8.821,0.19,0.21,0.0035,-0.0035,2988.0,2458389.2942323,0.00062395,-0.00062395,1.169,0.037,-0.038,1.76,0.17,-0.16,3.777939728,5.83523921e-06,-5.83523921e-06,0.02734,89.913,0.274,--,--,1,1,0.05103457908102151,12.61940060872384,399.3229614135901,4.2223523461028085,6.053946228618852,f1500w,full,3,7
LHS 1478,LHS 1478 b,18.2276,02h57m21.33s,+76d33m05.06s,8.767,0.24,0.25,0.01,-0.01,3381.0,2458786.75425,0.00042,-0.00042,1.242,0.051,-0.049,2.33,0.2,-0.2,1.9495378,4e-06,-4.1e-06,0.01848,87.452,--,--,--,1,1,0.045546069857697276,14.800207739484296,599.6495187649439,21.470754020600786,6.682832228969028,f1500w,full,2,7
LTT 3780,LTT 3780 b,21.9814,10h18m34.78s,-11d43m04.08s,8.204001,0.4,0.37,0.01,-0.01,3331.0,2458543.91281,0.00048,-0.00052,1.32,0.06,-0.06,2.47,0.24,-0.24,0.768377,1.4e-06,-1.4e-06,0.01203,85.9,0.064,0.075,-0.046,1,1,0.032707061524655315,13.89006163347313,890.7913247681156,104.55895980434461,5.901257760073781,f1500w,brightsky,2,12
TOI-1468,TOI-1468 b,24.7399,01h06m36.93s,+19d13m29.71s,8.497,0.34,0.34,0.01,-0.01,3496.0,2458765.68079,0.0007,-0.00069,1.28,0.038,-0.039,3.21,0.24,-0.24,1.8805136,2.4e-06,-2.6e-06,0.02102086,88.47,--,--,--,1,1,0.03451440360534036,19.197301030931996,677.9832124158797,35.08593869199558,8.410940260139574,f1500w,full,3,6
TOI-270,TOI-270 b,22.4793,04h33m39.86s,-51d57m26.62s,8.251,0.39,0.38,0.01,-0.01,3506.0,2458387.09505,0.00074,-0.00074,1.206,0.039,-0.039,1.58,0.26,-0.26,3.3601538,4.8e-06,-4.8e-06,0.03197,89.39,0.034,0.025,-0.025,1,1,0.029095982841969084,10.64431095709798,582.8619296384281,19.165497515761714,4.949765091265365,f1500w,brightsky,4,13


In [28]:
targ = sample_table[1]
print(targ['pl_name'])
c1 = SkyCoord(targ['rastr'], targ['decstr'], frame='icrs')
position_angle = 0 * u.deg # direction in "N-S" or "y" direction
separation = -15 * u.arcsec
c1_new = c1.directional_offset_by(position_angle, separation)

position_angle = -90 * u.deg # direction in "E-W" or "x" direction
separation = 5 * u.arcsec
c1_new = c1_new.directional_offset_by(position_angle, separation)

c1_new.to_string('hmsdms')

GJ 357 b


'09h36m01.43131904s -21d40m09.22997592s'

In [29]:
targ = sample_table[2]
print(targ['pl_name'])
c1 = SkyCoord(targ['rastr'], targ['decstr'], frame='icrs')
position_angle = 0 * u.deg # direction in "N-S" or "y" direction
separation = 8 * u.arcsec
c1_new = c1.directional_offset_by(position_angle, separation)

position_angle = -90 * u.deg # direction in "E-W" or "x" direction
separation = 8 * u.arcsec
c1_new = c1_new.directional_offset_by(position_angle, separation)

c1_new.to_string('hmsdms')

HD 260655 b


'06h37m09.41057321s +17d34m06.56995088s'